# Let's say we want to create a function that adds grocery items to a list...

In [1]:
def add_item(name, quantity, unit, grocery_list):
    additional_items = f'{name} ({quantity} {unit})'
    grocery_list.append(additional_items)

- *Why didn't we return the updated list?*
    - Because the list we passed in is a **mutable** object
        - Therefore, when the grocery list is appended to, the object at the existing memory address for `grocery_list` is updated
            - There's no need to return it
                - Additionally, **this is best practice**
                    - If no object is returned by the function, the user knows that the existing list was updated and there's no new object

____

**Aside**

- Let's show that returning the updated list would be redundant

In [2]:
def func(my_list):
    print(f'my_list = {my_list}, Address before append: {id(my_list)}')
    my_list.append(1)
    print(f'my_list = {my_list}, Address after append: {id(my_list)}')
    return my_list

In [3]:
my_list = []
my_list = func(my_list)
print(f'my_list = {my_list}, Address after return: {id(my_list)}')

my_list = [], Address before append: 2267269501832
my_list = [1], Address after append: 2267269501832
my_list = [1], Address after return: 2267269501832


- As we can see, all 3 addresses are the same
    - Therefore, no need to return the updated list object
    
___

- To show how the error can occur, let's redefine the function to return the list

In [4]:
def add_item(name, quantity, unit, grocery_list):
    additional_items = f'{name} ({quantity} {unit})'
    grocery_list.append(additional_items)
    return grocery_list

- Now, let's say we have two grocery stores (so we create two grocery lists)

In [5]:
list_1 = []
list_2 = []

- Now, we add items to our lists

In [6]:
add_item('banana', 2, 'units', list_1)
add_item('milk', 1, 'litre', list_1)

['banana (2 units)', 'milk (1 litre)']

In [7]:
add_item('python', 1, 'medium_rare', list_2)

['python (1 medium_rare)']

In [8]:
print(list_1, list_2)

['banana (2 units)', 'milk (1 litre)'] ['python (1 medium_rare)']


In [9]:
id(list_1), id(list_2)

(2267269520520, 2267270680008)

- As we can see, the two lists have distinct memory addresses

- Now, let's say we want to make it easier by setting an empty list as our default argument

In [10]:
def add_item(name, quantity, unit, grocery_list=[]):
    additional_items = f'{name} ({quantity} {unit})'
    grocery_list.append(additional_items)
    return grocery_list

In [11]:
del list_1, list_2

In [12]:
list_1 = add_item('banana', 2, 'units')
add_item('milk', 1, 'litre', list_1)

['banana (2 units)', 'milk (1 litre)']

In [13]:
list_2 = add_item('python', 1, 'medium_rare')

In [14]:
print(list_1, list_2)

['banana (2 units)', 'milk (1 litre)', 'python (1 medium_rare)'] ['banana (2 units)', 'milk (1 litre)', 'python (1 medium_rare)']


- Wtf is going on!?

In [15]:
id(list_1), id(list_2)

(2267270506312, 2267270506312)

- After our update, the two lists point to the same memory address

- *How can we fix this?*
    - Again, we use the `None` solution

In [16]:
def add_item(name, quantity, unit, grocery_list=None):
    grocery_list = grocery_list or []
    additional_items = f'{name} ({quantity} {unit})'
    grocery_list.append(additional_items)
    return grocery_list

In [17]:
del list_1, list_2

In [18]:
list_1 = add_item('banana', 2, 'units')
add_item('milk', 1, 'litre', list_1)

['banana (2 units)', 'milk (1 litre)']

In [19]:
list_2 = add_item('python', 1, 'medium_rare')

In [20]:
id(list_1), id(list_2)

(2267270519944, 2267270734664)

- Works as intended

____

# Now, let's say we're defining a factorial function...

In [21]:
def factorial(n):
    if n < 1:
        return 1
    else:
        print(f'calculating {n}!')
        return n * factorial(n - 1)

In [22]:
factorial(3)

calculating 3!
calculating 2!
calculating 1!


6

- Because the function calculates the factorial recursively, we run the print statement for 3, 2, and 1
    - Let's try running it a second time

In [23]:
factorial(3)

calculating 3!
calculating 2!
calculating 1!


6

- Again, it calculated the factorial recursively
    - *But since we already ran it, wouldn't it be more efficient to store the results?*
- Let's try redefining our function, except we'll use a mandatory `cache` argument

In [24]:
def factorial(n, *, cache):
    if n < 1:
        return 1
    elif n in cache:
        return cache[n]
    else:
        print(f'calculating {n}!')
        result = n * factorial(n - 1, cache=cache)
        cache[n] = result
        return result

- Before we execute the function, we need to define `cache` as an empty dictionary
    - *Why?*
        - Because it's a mandatory keyword parameter for our function

In [25]:
cache = {}

In [26]:
factorial(3, cache=cache)

calculating 3!
calculating 2!
calculating 1!


6

- *What's in the cache now?*

In [27]:
cache

{1: 1, 2: 2, 3: 6}

In [28]:
factorial(3, cache=cache)

6

- As we can see, none of the values were recalculated
    - They were all stored in the cache
- *What if we try a new one?*

In [29]:
factorial(5, cache=cache)

calculating 5!
calculating 4!


120

- As we can see, no extra code was run

- *How can we update this to make it easier to use?*
    - Changing `cache` from a mandatory argument to an optional one whose default value is an empty dictionary

In [30]:
def factorial(n, cache={}):
    if n < 1:
        return 1
    elif n in cache:
        print(id(cache))
        return cache[n]
    else:
        print(f'calculating {n}!')
        result = n * factorial(n - 1)
        cache[n] = result
        return result

In [31]:
del cache

In [32]:
factorial(3)

calculating 3!
calculating 2!
calculating 1!


6

In [33]:
factorial(3)

2267271158664


6

- Even though we didn't specify the updated `cache` object, it found it anyway
    - This is **because the default object we specified is mutable**
        - Therefore, it points to a memory address (whose contents can be updated i.e. mutated)!

- Let's see what's stored in the cache

In [34]:
cache

NameError: name 'cache' is not defined

- **Note**: this doesn't work because `cache` was never defined outside the function
    - To access it, we need to point to the memory address

In [35]:
import ctypes

- From our second call of `factorial(3)`, we know that the address is 2267271158664

In [36]:
address = 2267271158664
ctypes.cast(address, ctypes.py_object)

py_object({1: 1, 2: 2, 3: 6})

- There it is!